## Setup

In [1]:
import mechanicalsoup as ms
import pandas as pd
import numpy as np
import re

## Pull HTML

In [3]:
# get data as html
browser = ms.StatefulBrowser()
browser.open("https://www.boliga.dk/resultat?zipCodes=3450")
html = browser.get_current_page()

## Get Listings

In [4]:
boligs = []

for a in html.find_all('a', attrs={'href': re.compile("^/bolig/")}):
    url = "https://www.boliga.dk" + a.get('href')
    boligs.append(url)
    
boligs = np.unique(boligs)

# Get specific listing

In [9]:
boligs[0]

'https://www.boliga.dk/bolig/1701669/laehegnet_3_3450_alleroed'

In [52]:
def getStuff(url):
    
    browser = ms.StatefulBrowser()
    browser.open(url)
    soup = browser.get_current_page()

    inner_details = soup.find_all('div', attrs={'class': 'app-inner-details'})[0]
    spans = inner_details.select('span')
    spans = [item.get_text() for item in spans]
    
    stuff = {
        'url' : url,
        'pris' : spans[0],
        'size' : list(filter(re.compile('Boligstørrelse:').search, spans))[0]
    }
    
    return stuff

In [55]:
stuff_list = []
for b in boligs:
    s = getStuff(b)
    stuff_list.append(s)

In [56]:
stuff_list

[{'url': 'https://www.boliga.dk/bolig/1701669/laehegnet_3_3450_alleroed',
  'pris': ' 8.495.000\xa0kr. ',
  'size': ' Boligstørrelse: 232 m² '},
 {'url': 'https://www.boliga.dk/bolig/1706815/bjergvej_13_3450_alleroed',
  'pris': ' 4.445.000\xa0kr. ',
  'size': ' Boligstørrelse: 166 m² '},
 {'url': 'https://www.boliga.dk/bolig/1711938/guldborgvej_6_3450_alleroed',
  'pris': ' 6.250.000\xa0kr. ',
  'size': ' Boligstørrelse: 194 m² '},
 {'url': 'https://www.boliga.dk/bolig/1712322/bakkevej_16_3450_alleroed',
  'pris': ' 4.195.000\xa0kr. ',
  'size': ' Boligstørrelse: 159 m² '},
 {'url': 'https://www.boliga.dk/bolig/1713221/skytteengen_1_3450_alleroed',
  'pris': ' 3.345.000\xa0kr. ',
  'size': ' Boligstørrelse: 129 m² '},
 {'url': 'https://www.boliga.dk/bolig/1713803/gl_lyngevej_86b_3450_alleroed',
  'pris': ' 4.495.000\xa0kr. ',
  'size': ' Boligstørrelse: 170 m² '},
 {'url': 'https://www.boliga.dk/bolig/1713913/alleroed_park_19_st_-egen_terrasse-_3450_alleroed',
  'pris': ' 2.795.000\xa